<a href="https://colab.research.google.com/github/aquaqu-v246/AI-Products-All-In-One/blob/main/demo/VibeVoice_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader

NVIDIA A100-SXM4-40GB, 40960 MiB, 550.54.15


In [ ]:
%%writefile /content/my_transcript.txt
xxxx
xxxx

Writing /content/my_transcript.txt


In [ ]:
# -*- coding: utf-8 -*-
from google.colab import drive, files, runtime, auth
import os, shutil, pathlib, traceback, time, subprocess, sys

# ========= 可调参数 =========
OUTPUT_FILE = "/content/outputs/my_transcript_generated.wav"

# —— 保存到 Drive 的目录与“自定义保存名” —— #
DRIVE_DIR         = "/content/drive/MyDrive/vibevoice_outputs"
DRIVE_SAVE_NAME   = "case01.wav"   # 你想在网盘里保存的文件名（与源文件名可不同）
ADD_TIMESTAMP     = True           # True: 保存名后加 _YYYYmmdd-HHMMSS
AUTO_DEDUP        = True           # True: 若同名存在自动加 -1, -2…

WAIT_SECS         = 60             # 触发下载后等待多少秒再释放

# —— 从网上获取 —— #
REPO_URL          = "https://github.com/aquaqu-v246/VibeVoice.git"
REPO_DIR          = "/content/VibeVoice"
MODEL_ID          = "vibevoice/VibeVoice-7B"
MODEL_DIR         = "/content/models/VibeVoice-7B"

# —— 推理输入 —— #
TXT_PATH          = "/content/my_transcript.txt"
SPEAKERS          = ["Xinran", "Bowen"]   # 按需修改

# ========= 工具函数 =========
def sh(cmd: str, env=None):
    print(f"\n$ {cmd}")
    subprocess.run(cmd, shell=True, check=True, env=env)

def try_mount_drive(max_retries=3, sleep_sec=3):
    """挂载 Drive；成功 True，失败 False（不中断主流程）"""
    if os.path.isdir("/content/drive/MyDrive"):
        print("✅ Drive already mounted:", "/content/drive/MyDrive")
        return True
    try:
        print("🔐 Authenticating Google account ...")
        auth.authenticate_user()  # 弹出授权
    except Exception as e:
        print(f"⚠️ Auth failed: {e}")
    for i in range(1, max_retries + 1):
        try:
            print(f"📁 Mounting Google Drive (attempt {i}/{max_retries}) ...")
            drive.mount('/content/drive', force_remount=False)
            print("✅ Drive mounted at /content/drive")
            return True
        except Exception as e:
            print(f"⚠️ Mount failed: {e}")
            time.sleep(sleep_sec)
    print("❌ Drive mount failed after retries.")
    return False

def dedup_path(path):
    """若文件已存在，加 -1, -2… 避免覆盖"""
    if not os.path.exists(path):
        return path
    base, ext = os.path.splitext(path)
    k = 1
    while True:
        cand = f"{base}-{k}{ext}"
        if not os.path.exists(cand):
            return cand
        k += 1

def build_drive_dst(drive_dir, desired_name, add_timestamp, auto_dedup):
    base, ext = os.path.splitext(desired_name)
    if add_timestamp:
        ts = time.strftime("%Y%m%d-%H%M%S")
        desired_name = f"{base}_{ts}{ext}"
    dst = os.path.join(drive_dir, desired_name)
    if auto_dedup:
        dst = dedup_path(dst)
    return dst

# ========= 主流程 =========
mounted = False
try:
    # 0) 先挂载 Drive（需要授权）
    mounted = try_mount_drive()
    if mounted:
        pathlib.Path(DRIVE_DIR).mkdir(parents=True, exist_ok=True)

    # 1) 从网上获取仓库（始终在线克隆 / 更新）
    if not os.path.isdir(REPO_DIR):
        print("⬇️ Cloning repo ...")
        sh(f'git clone --branch main --depth 1 {REPO_URL} "{REPO_DIR}"')
    else:
        print("🔄 Repo exists. Updating ...")
        sh(f'git -C "{REPO_DIR}" pull --ff-only')

    # 2) 安装依赖
    sh(f'uv pip --quiet install --system -e "{REPO_DIR}"')
    print("✅ Dependencies installed.")

    # 3) 从网上获取模型（每次会话都下载/校验；HF CLI 会跳过已存在且一致的文件）
    pathlib.Path(MODEL_DIR).mkdir(parents=True, exist_ok=True)
    sh(f'HF_XET_HIGH_PERFORMANCE=1 hf download {MODEL_ID} --local-dir "{MODEL_DIR}"')
    print("✅ Model ready.")

    # 4) 实时推理（继承父进程输出，tqdm 原地刷新）
    cmd = [
        sys.executable,
        f"{REPO_DIR}/demo/inference_from_file.py",
        "--model_path", MODEL_DIR,
        "--txt_path", TXT_PATH,
        "--speaker_names", *SPEAKERS
    ]
    print("\n$ " + " ".join(cmd))
    subprocess.run(cmd, check=True)

    # 5) 确认输出
    if not os.path.exists(OUTPUT_FILE):
        raise FileNotFoundError(f"Output file not found: {OUTPUT_FILE}")
    print(f"✅ Found output: {OUTPUT_FILE}")

    # 6) 保存到 Drive（自定义保存名 + 可选时间戳/避重）
    drive_dst = None
    if mounted:
        desired_name = DRIVE_SAVE_NAME or os.path.basename(OUTPUT_FILE)
        drive_dst = build_drive_dst(DRIVE_DIR, desired_name, ADD_TIMESTAMP, AUTO_DEDUP)
        shutil.copy2(OUTPUT_FILE, drive_dst)
        if os.path.getsize(OUTPUT_FILE) != os.path.getsize(drive_dst):
            raise RuntimeError("Size mismatch after copy to Drive.")
        print(f"✅ Saved to Drive as: {drive_dst}")
    else:
        print("⚠️ Drive not mounted; skip saving to Drive.")



except Exception:
    traceback.print_exc()

finally:
    try:
        if mounted:
            drive.flush_and_unmount()  # 刷新/卸载 Drive
    except Exception:
        pass
    runtime.unassign()  # 释放 GPU/断开运行时


🔐 Authenticating Google account ...
📁 Mounting Google Drive (attempt 1/3) ...
Mounted at /content/drive
✅ Drive mounted at /content/drive
⬇️ Cloning repo ...

$ git clone --branch main --depth 1 https://github.com/aquaqu-v246/VibeVoice.git "/content/VibeVoice"

$ uv pip --quiet install --system -e "/content/VibeVoice"
✅ Dependencies installed.

$ HF_XET_HIGH_PERFORMANCE=1 hf download vibevoice/VibeVoice-7B --local-dir "/content/models/VibeVoice-7B"
✅ Model ready.

$ /usr/bin/python3 /content/VibeVoice/demo/inference_from_file.py --model_path /content/models/VibeVoice-7B --txt_path /content/my_transcript.txt --speaker_names Xinran Bowen
✅ Found output: /content/outputs/my_transcript_generated.wav
✅ Saved to Drive as: /content/drive/MyDrive/vibevoice_outputs/case01_20250913-164238.wav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ Browser download triggered (async).
⏳ Waiting 60s before releasing resources …
